## Estruturas de delay

#### FIR comb, IIR comb, universal (?), Natural sounding comb filter(?)

In [1]:
%run -i widgets.ipynb
%run -i 2_1introducao.ipynb
from math import tan, pi, cos, sin

def run_filter(file_name, filter_function, gain, delay_length):
    # leitura do arquivo WAV
    sample_rate, input_signal = read_audio_file(file_name)
    # aplicação do filtro
    if filter_function == 'fircomb':
        filtered_data, w, h = fircomb(input_signal, gain, delay_length)
    else:
        filtered_data, w, h = iircomb(input_signal, gain, delay_length)

    plot_filter(0, sample_rate, w, h)
   
    write_audio_file(filtered_data, sample_rate, 'output_file_comb.wav')

    %matplotlib ipympl
    plot_signal(input_signal, sample_rate, 'Sinal de Entrada')
    plot_signal(filtered_data, sample_rate, 'Sinal de Saída')
 

    display_audio(file_name, 'output_file_comb.wav')


In [2]:

def fircomb(input_signal, gain, delay_length):
    y = [0] * len(input_signal)
    x = input_signal
    g = gain
    delay_line = np.zeros(delay_length)
    for n in range(len(input_signal)):
        y[n] = x[n] + g * delay_line[delay_length - 1]
        delay_line[1:] = delay_line[:-1]
        delay_line[0] = x[n]
    w, h = freqz(y, input_signal, fs=44100)

    return y, w, h

def iircomb(input_signal, gain, delay_length):
    y = [0] * len(input_signal)
    x = input_signal
    g = gain
    delay_line = np.zeros(delay_length)
    for n in range(len(input_signal)):
        y[n] = x[n] + g * delay_line[delay_length - 1]
        delay_line[1:] = delay_line[:-1]
        delay_line[0] = y[n]
    w, h = freqz(y, input_signal, fs=44100)

    return y, w, h
    

#rodar o filtro
clear_output()
out1.clear_output()
out2.clear_output()
out3.clear_output()
run_filter('batida.wav', 'fircomb', 0.5, 2000)
widgets_delay()

Dropdown(description='Filter type:', options=('fircomb', 'iircomb'), value='fircomb')

Text(value='batida.wav', continuous_update=False, description='Nome do arquivo')

FloatSlider(value=0.5, continuous_update=False, description='Ganho', max=1.0, min=-1.0)

Text(value='2000', continuous_update=False, description='Tamanho do delay')

Button(button_style='success', description='Atualizar', style=ButtonStyle())